# Fancy Decorators

In the second part of this tutorial, we’ll explore more advanced features, including how to use the following:
    
* [1. Several decorators on one function](#several_decorators_on_one_function)
* [2. Decorators with arguments](#decorators_with_arguments)
* [3. Decorators that can optionally take arguments](#decorators_with_optional_arguments)
* [4. Stateful decorators](#stateful_decorators)
* [5. Classes as decorators](#classes_as_decorators)

<a class="anchor" id="several_decorators_on_one_function"></a>
## Several decorators on one function

You can apply several decorators to a function by stacking them on top of each other:

In [1]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [2]:
@do_twice
@debug
def greet(name):
    print(f"Hello {name}")

Think about this as the decorators being executed in the order they are listed. In other words, `@debug` calls `@do_twice`, which calls `greet()`, or `debug(do_twice(greet()))`:

In [3]:
greet("Ali")

Calling greet('Ali')
Hello Ali
'greet' returned None
Calling greet('Ali')
Hello Ali
'greet' returned None


Observe the difference if we change the order of `@debug` and `@do_twice`:

In [4]:
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

In this case, `@do_twice` will be applied to `@debug` as well:

In [5]:
greet("Ali")

Calling greet('Ali')
Hello Ali
Hello Ali
'greet' returned None


<a class="anchor" id="decorators_with_arguments"></a>
## Decorators With Arguments

Sometimes, it’s useful to pass arguments to your decorators. For instance, `@do_twice` could be extended to a `@repeat(num_times)` decorator. The number of times to execute the decorated function could then be given as an argument.

This would allow you to do something like this:

```python
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

>>> greet("World")
Hello World
Hello World
Hello World
Hello World
```

So far, the name written after the `@` has referred to a function object that can be called with another function. To be consistent, you then need `repeat(num_times=4)` to return a function object that can act as a decorator. Luckily, you already know how to return functions! In general, you want something like the following:

In [11]:
def repeat(num_times):
    def decorator_repeat(func):
        ...  # Create and return a wrapper function
    return decorator_repeat

Typically, the decorator creates and returns an inner wrapper function, so writing the example out in full will give you an inner function within an inner function. While this might sound like the programming equivalent of the Inception movie, we’ll untangle it all in a moment:

In [8]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

It looks a little messy, but we have only put the same decorator pattern you have seen many times by now inside one additional def that handles the arguments to the decorator. Let’s start with the innermost function:

```python
def wrapper_repeat(*args, **kwargs):
    for _ in range(num_times):
        value = func(*args, **kwargs)
    return value
```

This `wrapper_repeat()` function takes arbitrary arguments and returns the value of the decorated function, `func()`. This wrapper function also contains the loop that calls the decorated function num_times times. This is no different from the earlier wrapper functions you have seen, except that it is using the `num_times` parameter that must be supplied from the outside.

One step out, you’ll find the decorator function:
```python
def decorator_repeat(func):
    @functools.wraps(func)
    def wrapper_repeat(*args, **kwargs):
        ...
    return wrapper_repeat
```

Again, `decorator_repeat()` looks exactly like the decorator functions you have written earlier, except that it’s named differently. That’s because we reserve the base name—`repeat()`—for the outermost function, which is the one the user will call.

As you have already seen, the outermost function returns a reference to the decorator function:

```python
def repeat(num_times):
    def decorator_repeat(func):
        ...
    return decorator_repeat
```

There are a few subtle things happening in the repeat() function:

- Defining `decorator_repeat()` as an inner function means that `repeat()` will refer to a function object—decorator_repeat. Earlier, we used repeat without parentheses to refer to the function object. The added parentheses are necessary when defining decorators that take arguments.
- The `num_times` argument is seemingly not used in `repeat()` itself. But by passing num_times a closure is created where the value of `num_times` is stored until it will be used later by `wrapper_repeat()`.

With everything set up, let’s see if the results are as expected:

In [9]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")
    
# TODO: Add figure here.
# 1. repeat
# 2. repeat(num_times=4) ---> returns decorator
# 3. decorator ---> returns wrapper

In [10]:
greet("Ali")

Hello Ali
Hello Ali
Hello Ali
Hello Ali


<a class="anchor" id="decorators_with_optional_arguments"></a>
## Decorators that Can Optionally Take Arguments

With a little bit of care, you can also define decorators that can be used both with and without arguments. Most likely, you don’t need this, but it is nice to have the flexibility.

As you saw in the previous section, when a decorator uses arguments, you need to add an extra outer function. The challenge is for your code to figure out if the decorator has been called with or without arguments.

Since the function to decorate is only passed in directly if the decorator is called without arguments, the function must be an optional argument. This means that the decorator arguments must all be specified by keyword. You can enforce this with the special `*` syntax, which means that all following parameters are keyword-only:

```python
def name(_func=None, *, kw1=val1, kw2=val2, ...):  # 1
    def decorator_name(func):
        ...  # Create and return a wrapper function.

    if _func is None:
        return decorator_name                      # 2
    else:
        return decorator_name(_func)               # 3
```

Here, the `_func` argument acts as a marker, noting whether the decorator has been called with arguments or not:

1. If `name` has been called without arguments, the decorated function will be passed in as `_func`. If it has been called with arguments, then `_func` will be `None`, and some of the keyword arguments may have been changed from their default values. The `*` in the argument list means that the remaining arguments can’t be called as positional arguments.
2. In this case, the decorator was called with arguments. Return a decorator function that can read and return a function.
3. In this case, the decorator was called without arguments. Apply the decorator to the function immediately.

Using this boilerplate on the `@repeat` decorator in the previous section, you can write the following:

In [4]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

Compare this with the original `@repeat`. The only changes are the added `_func` parameter and the `if-else` at the end.

These examples show that `@repeat` can now be used with or without arguments:

In [8]:
@repeat
def say_whee():
    print("Whee!")

# 1
# input: function
# say_whee = repeat(say_whee)

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

# 2
# input: keyword argument
# greet = repeat(num_times=3)(greet)

Recall that the default value of `num_times` is `2`:

In [9]:
say_whee()

Whee!
Whee!


In [10]:
greet("Ali")

Hello Ali
Hello Ali
Hello Ali


<a class="anchor" id="stateful_decorators"></a>
## Stateful Decorators

Sometimes, it’s useful to have **a decorator that can keep track of state**. As a simple example, we will create a decorator that counts the number of times a function is called.

In the next section (_Classes as Decorators_), you will see how to use classes to keep state. But in simple cases, you can also get away with using function attributes:

In [50]:
import functools


def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    
    # set attribute before return
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls


@count_calls
def say_whee():
    print("Whee!")

The state—the number of calls to the function—is stored in the function attribute `.num_calls` on the wrapper function. Here is the effect of using it:

In [51]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [52]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [53]:
say_whee.num_calls

2

<a class="anchor" id="classes_as_decorators"></a>
## Classes as Decorators

The typical way to maintain state is by using classes. In this section, you’ll see how to rewrite the `@count_calls` example from the previous section **using a class as a decorator**.

Recall that the decorator syntax `@my_decorator` is just an easier way of saying `func = my_decorator(func)`. Therefore, if `my_decorator` is a class, it needs to take func as an argument in its `.__init__()` method. Furthermore, the class instance needs to be callable so that it can stand in for the decorated function.

For a class instance to be callable, you implement the special `.__call__()` method:

In [78]:
class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f"Current count is {self.count}")

The `.__call__()` method is executed each time you try to call an instance of the class:

In [50]:
counter = Counter()

In [51]:
counter()

Current count is 1


In [52]:
counter()

Current count is 2


Therefore, a typical implementation of a decorator class needs to implement `.__init__()` and `.__call__()`:

In [11]:
import functools


class CountCalls:
    def __init__(self, func, *, num_times=2):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)


@CountCalls
def say_whee():
    print("Whee!")
    
# @Counter is identical to:
# say_hi = CountCalls(say_hi)

The `.__init__()` method must store a reference to the function and can do any other necessary initialization. The `.__call__()` method will be called instead of the decorated function. It does essentially the same thing as the `wrapper() `function in our earlier examples. Note that you need to use the `functools.update_wrapper()` function instead of `@functools.wraps`.

This `@CountCalls` decorator works the same as the one in the previous section:

In [54]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [55]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [56]:
say_whee()

Call 3 of 'say_whee'
Whee!


In [57]:
say_whee.__name__

'say_whee'